In [1]:
import pandas as pd
import os
import git

In [2]:
def get_git_root(path):
    git_repo = git.Repo(path, search_parent_directories=True)
    git_root = git_repo.git.rev_parse("--show-toplevel")
    return git_root

txn_date_col_name = "txn_date"
txn_description_col_name = "txn_description"
txn_amount_col_name = "txn_amount"
txn_category_col_name = "category"
txn_subcat_col_name = "sub_category"
processed_txn_filename = "categorized_txn.csv"
bank_col_name = "bank"
id_col_name = "unique_id"
txn_strftime_format = "%Y-%m-%d"

data_root_dir_append = "data"
data_raw_dir_append = "data/raw"
data_interim_dir_append = "data/interim"
data_processed_dir_append = "data/processed"

processed_txn_filename = "categorized_txn.csv"

In [3]:
git_root_path = get_git_root(os.getcwd())
interim_data_path = os.path.join(git_root_path, data_interim_dir_append)
interim_txn_path = os.path.join(interim_data_path, processed_txn_filename)

interim_txn_df = pd.read_csv(interim_txn_path, index_col= id_col_name)
interim_txn_df[txn_date_col_name] = pd.to_datetime(interim_txn_df[txn_date_col_name], format=txn_strftime_format)
#interim_txn_df[txn_date_col_name] = pd.to_datetime(interim_txn_df[txn_date_col_name], format="%m/%d/%y")

In [ ]:
# interim_txn_df[txn_date_col_name] = interim_txn_df.apply(
#     lambda x: x[txn_date_col_name].strftime(txn_strftime_format),
#     axis = 1
# )

# interim_txn_df = interim_txn_df.sort_values("txn_date", ascending=False)
# interim_txn_df.to_csv(interim_txn_path)

In [4]:
interim_txn_df

,category,sub_category,txn_amount,txn_date,txn_description
unique_id,,,,,
usaa_checking_20191217_21.73,Vanlife,Auto Parts,-21.73,2019-12-17,AMZN Mktp US*XA2802ID3 AMZN.COM BILLWA
usaa_checking_20191217_8.0,Gifts and Donations,Holiday/Birthday,-8.00,2019-12-17,TST* STIR COOKING SCHOOLTDENVER CO
usaa_checking_20191217_4.2,Dining Out,Coffee,-4.20,2019-12-17,SQ *ALLEGRO COFFEE DENVER CO
usaa_checking_20191216_53.2,Gifts and Donations,Holiday/Birthday,-53.20,2019-12-16,KiwiCo Inc. 800-7144828 CA
usaa_checking_20191216_23.65,Dining Out,Bars,-23.65,2019-12-16,WILDWOOD VAIL CO
...,...,...,...,...,...
usaa_checking_20191017_40.53,Vanlife,Accessories,-40.53,2019-10-17,AMZN Mktp US*T99F66V83 AMZN.COM BILLWA
usaa_checking_20191016_7.95,Groceries,Non-Food,-7.95,2019-10-16,SKIN CO. HTTPSWWW.LUMICA
usaa_checking_20191016_8.0,Dining Out,Bars,-8.00,2019-10-16,RUSTY MELON ERIE CO


In [ ]:
processed_txn_df = interim_txn_df.copy()

In [ ]:
processed_txn_df

In [ ]:
processed_txn_df['year'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].year
    , axis = 1
)

processed_txn_df['month'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].month
    , axis = 1
)

processed_txn_df['day'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].day
    , axis = 1
)

processed_txn_df['weekday'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].strftime("%A")
    , axis = 1
)

# get expenses to be positive
processed_txn_df[txn_amount_col_name] = processed_txn_df.apply(
    lambda x: 
    -1 * x[txn_amount_col_name] if x[txn_category_col_name] != 'Salary' else x[txn_amount_col_name]
    , axis = 1
)

In [ ]:
processed_txn_df

In [ ]:
cat_detail_df = processed_txn_df.groupby([txn_category_col_name, 'year', 'month']).agg(
    sum_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=sum),
    num_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=pd.Series.nunique)
).reset_index()

subcat_detail_df = processed_txn_df.groupby([txn_category_col_name, txn_subcat_col_name, 'year', 'month']).agg(
    sum_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=sum),
    num_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=pd.Series.nunique)
).reset_index()

cat_detail_df['avg_per_txn'] = cat_detail_df['sum_txns'] / cat_detail_df['num_txns']
subcat_detail_df['avg_per_txn'] = subcat_detail_df['sum_txns'] / subcat_detail_df['num_txns']

In [ ]:
subcat_detail_df
#subcat_detail_df.to_csv('./subcat_detail.csv', index= False)
# subcat_detail_df.pivot(index='month', columns='category', values='sum_txns').plot(kind='bar')

In [ ]:
subcat_detail_df[[txn_category_col_name, txn_subcat_col_name]].drop_duplicates().reset_index(drop=True)

In [ ]:
cat_specific_df = month_grouped_df[month_grouped_df['category'] == 'Food and Dining']
#example_df = month_grouped_df[month_grouped_df['sub_category'] == "Auto - Gas"]
#month_grouped_df[month_grouped_df['category'] == "Auto and Transport"]
cat_specific_df

In [ ]:
cat_specific_df.pivot(index='month', columns='sub_category', values='sum_txns').plot(kind='bar')